<a href="https://colab.research.google.com/github/FairyAmp/linear-systems-analysis/blob/main/ode_linear_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ODE Linear Systems Solver with Phase Portrait

A linear system is the matrix multiplication: $\mathbf{\dot{x}} = A\mathbf{x}$.

In the traditional linear algebra sense, $A$ is simply a matrix. More specifically, it is a an operator. The current state is represented by the vector $\mathbf{x}$, which is defined with respect to the time variable $t$.



## Imports

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.linalg import expm
import matplotlib.pyplot as plt

#optional for sliders
import ipywidgets as widgets
from IPython.display import display

print(f"NumPy version: {np.__version__}")

# ODE Solver

odeint and expm

## odeint

In [ ]:
def solve_lin_ode_numeric(A, x_0, t):

  # derivative function (locally)
  def system_dynamics(x,t):
    return A @ x

  # call solver
  solution = odeint(system_dynamics, x_0, t)

  return solution, t

## expm

In [ ]:
def solve_lin_ode_exact(A, x_0, t):
  x = np.array([expm(A * t_i) @ x_0 for t_i in t])
  return x, t

## Comparison Test and Plots

In [ ]:
t = np.linspace(0, 10, 100)
A = np.array([[-1, 2], [-2, -1]])
x_0 = [1, 1]

sol_num, t_num = solve_lin_ode_numeric(A, x_0, t)
sol_exact, t_ex = solve_lin_ode_exact(A , x_0, t)

# phase portrait
plt.plot(sol_num[:,0], sol_num[:, 1], 'b', label='Numerical (odeint)', lw=4, alpha=.7)
plt.plot(sol_exact[:,0], sol_exact[:,1], 'ro--', label='Exact (expm)')

### Print

Numerical vs exact

In [ ]:
print("t:", t[:5], "...", t[-5:])
print("Numerical solution (first 5):\n", sol_num[:5])
print("Exact solution (first 5):\n", sol_exact[:5])

# error check
err = np.max(np.linalg.norm(sol_num - sol_exact, axis=1))
print("Max error:", err)

# Consider the following linear system

$$ \mathbf{\dot{x}} = \begin{pmatrix}
1 & -4\\
4 & -7
\end{pmatrix} \mathbf{x}, \quad
\mathbf{x}(0) = \begin{pmatrix} 3\\ 2 \end{pmatrix}$$

In [ ]:
A = np.array([[1, -4],
              [4, -7]])
x_0 = [3, 2]

t = np.linspace(0, 10, 1000)
sol_num, t_num = solve_lin_ode_numeric(A, x_0, t)
sol_exact = np.array([expm(A*ti) @ x_0 for ti in t])

# phase portrait
plt.plot(sol_num[:,0], sol_num[:, 1], 'b', label='Numerical (odeint)', lw=4, alpha=.7)
plt.plot(sol_exact[:,0], sol_exact[:,1], 'y--', label='Exact (expm)')

## Plots

In [ ]:
plt.plot(sol_num[:,0], t, 'b', label='x1 numeric')
plt.plot(sol_num[:,1], t, 'g', label='x2 numeric')
plt.plot(sol_exact[:,0], t, 'r--', label='x1 exact')
plt.plot(sol_exact[:,1], t, 'r-.', label='x2 exact')

## Comparison with actual solution

$$x(t)=2e^{- 3t} \begin{pmatrix} 1 \\ 1 \end{pmatrix} + e^{-3t} \begin{pmatrix} 4t + 1 \\ 4t \end{pmatrix}$$

In [ ]:
# time array
t = np.linspace(0, 5, 500)

# compute x(t)
x_1 = 2 * np.exp(-3*t)* 1 + np.exp(-3*t) * (4*t + 1) # first component
x_2 = 2 * np.exp(-3*t)* 1 + np.exp(-3*t) * (4*t) # second component

# stack into array
# shape (len(t), 2)
x = np.vstack([x_1, x_2]).T

# plot
plt.plot(x_1, t, label='$x_1(t)$')
plt.plot(x_2, t, label='$x_2(t)$')
plt.xlabel('$x(t)$')
plt.ylabel('$t$')
plt.title('$x(t) = 2 e^{-3t} [1,1] + e^{-3t} [4t+1, 4t]$')
plt.legend()
plt.grid(True)
plt.show()

## Phase Portrait $x_1$ vs $x_2$

In [ ]:
plt.plot(x_1, x_2, 'b', lw=2)
plt.xlabel('$x_1(t)$')
plt.ylabel('$x_2(t)$')
plt.title('Phase portrait of $x(t) = 2 e^{-3t}[1,1] + e^{-3t}[4t+1, 4t]$')
plt.grid(True)
plt.axis('equal')
plt.show()

## Phase Portrait with Direction of Motion

In [ ]:
# phase portrait
plt.plot(x_1, x_2, 'b', lw=2, label='Trajectory')

# add arrows along the curve
skip = 20  # spacing between arrows
for i in range(0, len(t)-skip, skip):
    plt.arrow(x_1[i], x_2[i],
              x_1[i+skip]-x_1[i], x_2[i+skip]-x_2[i],
              shape='full', lw=0, length_includes_head=True, head_width=0.1, color='r')

plt.xlabel('$x_1(t)$')
plt.ylabel('$x_2(t)$')
plt.title('Phase portrait with direction arrows')
plt.grid(True)
plt.axis('equal')
plt.legend()
plt.show()

In [ ]:
# function for dx/dt
def f(x,t):
  return A @ x

# grid for vector field
x_1_vals = np.linspace(-5, 5, 20)
x_2_vals = np.linspace(-5, 5, 20)
X1, X2 = np.meshgrid(x_1_vals, x_2_vals)

# compute dx/dt at each grid point
U = A[0,0]*X1 + A[0,1]*X2  # dx_1/dt
V = A[1,0]*X1 + A[1,1]*X2  # dx_2/dt

# normalize vectors for better visualization
N = np.sqrt(U**2 + V**2) + 1e-12
U2, V2 = U/N, V/N

plt.figure(figsize=(7,7))

# plot vector field
plt.quiver(X1, X2, U2, V2, color='gray')

# trajectories from multiple initial conditions
initial_conditions = [[3,2], [1,0], [-2,1], [0,-3], [-1,-2]]
t = np.linspace(0, 5, 500)

for x_0 in initial_conditions:
    sol = odeint(f, x_0, t)
    plt.plot(sol[:,0], sol[:,1], lw=2)
    # add arrows along trajectory
    skip = 25
    for i in range(0, len(t)-skip, skip):
        plt.arrow(sol[i,0], sol[i,1],
                  sol[i+skip,0]-sol[i,0], sol[i+skip,1]-sol[i,1],
                  shape='full', lw=0, length_includes_head=True, head_width=0.12, color='r')

# overlay the analytical solution: x(t) = 2e^{-3t}[1,1] + e^{-3t}[4t+1, 4t]
#x1_analytical = 2*np.exp(-3*t)*1 + np.exp(-3*t)*(4*t + 1)
#x2_analytical = 2*np.exp(-3*t)*1 + np.exp(-3*t)*(4*t)
#plt.plot(x1_analytical, x2_analytical, 'b--', lw=1, label='Analytical solution')

plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Phase Portrait with Vector Field and Trajectories')
plt.grid(True)
plt.axis('equal')
plt.show()

The blue line above is the phase portrait for when $x_0 = \begin{pmatrix} 3 \\ 2 \end{pmatrix}$.

## Phase Portrait with trace det and nullclines

In [ ]:
def plot_enhanced_phase_portrait(A, x0_list):
    # --- 1. Math Analysis ---
    tr = np.trace(A)
    det = np.linalg.det(A)
    disc = tr**2 - 4*det
    eigenvals = np.linalg.eigvals(A)

    # Classification Logic
    if det < 0:
        kind = "Saddle Point"
    elif disc < 0:
        kind = "Center" if tr == 0 else ("Stable Spiral" if tr < 0 else "Unstable Spiral")
    else:
        kind = "Stable Node" if tr < 0 else "Unstable Node"

    # --- 2. Setup Plot ---
    fig, ax = plt.subplots(figsize=(8, 8))
    x_range = np.linspace(-5, 5, 20)
    y_range = np.linspace(-5, 5, 20)
    X, Y = np.meshgrid(x_range, y_range)

    U = A[0,0]*X + A[0,1]*Y
    V = A[1,0]*X + A[1,1]*Y

    # Background Flow (gray, semi-transparent)
    strm = ax.streamplot(X, Y, U, V, color=(0.5,0.5,0.5,0.3), density=1.5)

    # --- 3. Add Nullclines ---
    if A[0,1] != 0:
        ax.plot(x_range, -(A[0,0]/A[0,1])*x_range, 'r--', alpha=0.5, label=r'$\dot{x}_1=0$')
    if A[1,1] != 0:
        ax.plot(x_range, -(A[1,0]/A[1,1])*x_range, 'g--', alpha=0.5, label=r'$\dot{x}_2=0$')

    # --- 4. Plot Trajectories ---
    t = np.linspace(0, 10, 1000)
    for x0 in x0_list:
        sol = odeint(lambda x, t: A @ x, x0, t)
        ax.plot(sol[:,0], sol[:,1], lw=2)
        # optional arrows along trajectory
        skip = 25
        for i in range(0, len(t)-skip, skip):
            ax.arrow(sol[i,0], sol[i,1],
                     sol[i+skip,0]-sol[i,0], sol[i+skip,1]-sol[i,1],
                     shape='full', lw=0, length_includes_head=True, head_width=0.12, color='r')

    # --- 5. Styling ---
    ax.axhline(0, color='black', lw=1.5)
    ax.axvline(0, color='black', lw=1.5)
    ax.set_xlim([-5, 5])
    ax.set_ylim([-5, 5])
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title(rf"Type: {kind}\n$\lambda_1, \lambda_2 = {eigenvals[0]:.2f}, {eigenvals[1]:.2f}$")
    ax.legend()
    ax.grid(True)
    plt.show()

# --- Example Call ---
A_matrix = np.array([[-1, 4], [-1, -5]])
initial_pts = [[3,2], [1,0], [-2,1]]
plot_enhanced_phase_portrait(A_matrix, initial_pts)


# Testing Widgets

Testing out widgets for interactive phase portraits.

In [ ]:
def interactive_plot(a, b, c, d):
    # 1. Re-define the matrix based on sliders
    A = np.array([[a, b],
                  [c, d]])

    # 2. Setup the grid
    x = np.linspace(-5, 5, 20)
    y = np.linspace(-5, 5, 20)
    X1, X2 = np.meshgrid(x, y)

    # 3. Vector Field
    U = A[0,0]*X1 + A[0,1]*X2
    V = A[1,0]*X1 + A[1,1]*X2

    # 4. Eigenvalue analysis for the title
    eigenvals = np.linalg.eigvals(A)

    # 5. Plotting
    plt.figure(figsize=(8, 8))

    # Streamplot handles normalization and arrows automatically
    speed = np.sqrt(U**2 + V**2) + 1e-9
    plt.streamplot(X1, X2, U, V, color=speed, cmap='viridis', density=1)

    # Add a sample trajectory from a fixed start point
    t = np.linspace(0, 10, 500)
    sol = odeint(lambda x, t: A @ x, [2, 2], t)
    plt.plot(sol[:,0], sol[:,1], 'r-', lw=3, label='Sample Trajectory from (2,2)')

    # Visual Polish
    plt.axhline(0, color='black', lw=1)
    plt.axvline(0, color='black', lw=1)
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.title(f"Eigenvalues: {eigenvals[0]:.2f}, {eigenvals[1]:.2f}")
    plt.legend()
    plt.show()

# 6. Create the sliders
# Format: widgets.FloatSlider(value, min, max, step, description)
w = widgets.interactive(interactive_plot,
    a=widgets.FloatSlider(value=-1, min=-5, max=5, step=0.1, description='a'),
    b=widgets.FloatSlider(value=2,  min=-5, max=5, step=0.1, description='b'),
    c=widgets.FloatSlider(value=-2, min=-5, max=5, step=0.1, description='c'),
    d=widgets.FloatSlider(value=-1, min=-5, max=5, step=0.1, description='d')
)

display(w)